In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import numpy as np
import torch


poison_per_category = 100
poisoners = ['desk', 'palace', 'necklace', 'balloon', 'pillow', 
             'candle', 'pizza', 'umbrella', 'television', "baseball", 
             "ice cream", "suit", 'mountain', 'beach', 'plate',
             'orange']
full_poison_range = poison_per_category * len(poisoners)

def plot_poison_distribution(file_path, poison_category='full', filter_ratios=[0.1, 0.15, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]):
    # df = pd.read_csv(file_path, sep='\t', header=None)
    t = torch.load(file_path).cpu().numpy()
    df = pd.DataFrame(t, columns=None)
    mean_similarity = df[1].mean()
    orig_len = len(df)

    if poison_category == 'full':
        condition = df[0] >100000
    elif poison_category == 'less':
        condition = df[0] < (full_poison_range // 2)
    else:
        condition = (df[0] >= poison_per_category * poisoners.index(poison_category) \
                        and df[0] < poison_per_category * (poisoners.index(poison_category)+1))
    df = df[condition]
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 6))

    n, bins, patches = ax1.hist(df.index.tolist(), bins=50, color='blue', alpha=0.5)
    ax1.xaxis.set_major_locator(plt.MaxNLocator(integer=True))
    ax1.set_title('Poison Rank Distribution')
    ax1.set_xlabel('Poison Rank')
    ax1.set_ylabel('Frequency')
    
    comments = []
    for ratio in filter_ratios:
        unfiltered_poison_num = (df.index < orig_len * ratio).sum()
        comments.append('poison num at top %f: %d'%(ratio, unfiltered_poison_num))
    comment_x = np.argmax(n)
    comment_y = np.max(n) 
    ax1.text(comment_x, comment_y, '\n'.join(comments), fontsize=12, ha='center')


    ax2.hist(df[1].tolist(), bins=30, color='green', alpha=0.5)
    ax2.invert_xaxis()
    ax2.set_title('Poison Similarity Distribution')
    ax2.set_xlabel('Poison Similarity')
    ax2.set_ylabel('Frequency')
    ax2.axvline(mean_similarity, color='red', linestyle='--', label='mean_similarity')
    ax2.legend()

    plt.tight_layout()
    plt.savefig('post_pretraining_analysis/dist_%s_%s.png' \
                %(re.search(r"/([^/]+).pt", file_path).group(1), poison_category))
    plt.close()

/home/hyang/deadclip/CyCLIP/env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
plot_poison_distribution("/home/hyang/NNCLIP/CyCLIP/jigao_indices/ablation_cross_inmodal_update37.pt")

In [21]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import numpy as np
import torch
from scipy.interpolate import make_interp_spline
from matplotlib.lines import Line2D  # I
from matplotlib import ticker

In [5]:
poison_per_category = 100
poisoners = ['desk', 'palace', 'necklace', 'balloon', 'pillow', 
             'candle', 'pizza', 'umbrella', 'television', "baseball", 
             "ice cream", "suit", 'mountain', 'beach', 'plate',
             'orange']
full_poison_range = poison_per_category * len(poisoners)

In [36]:
def plot_clean_n_poison_similarity_distribution(file_path, poison_category='full'):
    if file_path[-2:] == 'pt':
        # import pdb
        # pdb.set_trace()
        t = torch.load(file_path).cpu().numpy()
        df = pd.DataFrame(t, columns=None)
    else:
        df = pd.read_csv(file_path, sep='\t', header=None)

    if poison_category == 'full':
        poison_condition = df[0] < full_poison_range
        clean_condition = df[0] >= full_poison_range
    elif poison_category == 'less':
        poison_condition = df[0] < (full_poison_range // 2)
        clean_condition = df[0] >= (full_poison_range // 2)
    else:
        condition = (df[0] >= poison_per_category * poisoners.index(poison_category) \
                        and df[0] < poison_per_category * (poisoners.index(poison_category)+1))
    df_poison = df[poison_condition]
    df_clean = df[clean_condition]
    
    fig, ax = plt.subplots(figsize=(8, 6))

    
    # n, bins, patches =  plt.hist(df_poison[1].tolist(), bins=30, color='blue', alpha=0.5)
    n, bins = np.histogram(df_poison[1].tolist(), bins=20, density=True)
    # n = n / n.max() * 1.1
    # Compute the midpoints of each bin
    bin_centers = 0.5 * (bins[:-1] + bins[1:])

    x_smooth = np.linspace(bin_centers.min(), bin_centers.max(), 100)  # Generate more points for a smoother curve
    y_smooth = make_interp_spline(bin_centers, n)(x_smooth)

    # Plot the smoothed curve
    # plt.plot(x_smooth, y_smooth, 'b-', linewidth=2)
    plt.fill_between(x_smooth, 0, y_smooth, alpha=0.3, color='blue')

    # n, bins, patches =  plt.hist(df_clean[1].tolist(), bins=30, color='green', alpha=0.5)
    n, bins = np.histogram(df_clean[1].tolist(), bins=30, density=True)
    # n = n / n.max() * 1.1
    # Compute the midpoints of each bin
    bin_centers = 0.5 * (bins[:-1] + bins[1:])
    x_smooth = np.linspace(bin_centers.min(), bin_centers.max(), 100)  # Generate more points for a smoother curve
    y_smooth = make_interp_spline(bin_centers, n)(x_smooth)

    # Plot the smoothed curve
    # plt.plot(x_smooth, y_smooth, 'g-', linewidth=2)
    plt.axvline(x=0.3, color='r', linestyle='--', linewidth=2)
    plt.fill_between(x_smooth, 0, y_smooth, alpha=0.3, color='green')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    plt.ylim(0, 5.5)
    plt.xlim(-0.4, 0.6)
    ax.set_ylabel('Probability Density', fontsize=30)  
    ax.set_xlabel('Cosine Similarity', fontsize=30) 
    # Create custom legend entries with colored boxes
    legend_elements = [
        Line2D([0], [0], color='blue', lw=10, alpha=0.3, label='Poison'),
        Line2D([0], [0], color='green', lw=10, alpha=0.3, label='Clean'),
    ]
    ax = plt.gca()
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.xaxis.set_major_locator(ticker.MultipleLocator(0.2))
    plt.xticks(fontsize=30)
    plt.yticks(fontsize=30)

    # Add legend with custom entries
    ax.legend(handles=legend_elements, handlelength=3, handleheight=3, fontsize=20)

    plt.tight_layout()

    if file_path[-2:] == 'pt':
        plt.savefig('post_pretraining_analysis/clean_poison_dist_%s_%s.png' \
                %(re.search(r"/([^/]+).pt", file_path).group(1), poison_category))
    else:
        plt.savefig('post_pretraining_analysis/clean_poison_dist_%s_%s.png' \
            %(re.search(r"/([^/]+).tsv", file_path).group(1), poison_category))
    plt.close()
# plot_clean_n_poison_similarity_distribution("/home/hyang/NNCLIP/CyCLIP/jigao_indices/clip_poison_num_compare_update1.tsv")
plot_clean_n_poison_similarity_distribution("/home/hyang/NNCLIP/CyCLIP/jigao_indices/NNCLIP_1M_100_16_w_NN_wo_intersection_update7.pt")

In [ ]:
import numpy as np
for i in range(5):
    a = np.load("/home/hyang/NNCLIP/clip_1M_100_poison_200_backdoor_{}_tiger.npz".format(i+1))
    arr = a["arr_0"][0]
    indices = np.argsort(arr)
    top_50_elements = arr[indices[-2000:]]
    print(top_50_elements.mean())

In [1]:
from sklearn.mixture import GaussianMixture
gmm = GaussianMixture(n_components=2, random_state=0,warm_start=False)

def fit_gmm_to_cos_sim(gmm, cos_sim):
    """
    Fits a Gaussian Mixture Model to the given loss values.

    Args:
    cos_sim (np.array): An array of loss values.
    n_components (int): The number of components for GMM. Default is 2.

    Returns:
    GaussianMixture: The fitted GMM model.
    np.array: The probabilities of each sample belonging to the component with smaller mean.
    """
    # Reshape cos_sim for GMM compatibility
    cos_sim = np.array(cos_sim).reshape(-1, 1)

    # Fit the GMM
    
    gmm.fit(cos_sim)

    # Predict probabilities
    probabilities = gmm.predict_proba(cos_sim)

    # Identify the component with the smaller mean
    larger_mean_index = np.argmax(gmm.means_)
    smaller_mean_index = np.argmin(gmm.means_)

    # Return the GMM model and probabilities of belonging to the component with smaller mean
    return gmm, probabilities[:, larger_mean_index], probabilities[:, smaller_mean_index]

In [6]:
import torch
import numpy as np
for i in [7]:
    epoch = i
    res = torch.load("/home/hyang/NNCLIP/CyCLIP/indices/SafeCLIP_3M_update{}.pt".format(epoch))
    cos_sim_threshold = 0.15
    cos_sim = res.T[1].cpu().numpy()
    index = res.T[0]
    gmm_model, clean_probabilities, poisoned_probabilities = fit_gmm_to_cos_sim(gmm, cos_sim)

    clean_mean = np.max(gmm.means_)
    # print("GMM Means:", gmm_model.means_)
    # print("Clean Probabilities:", clean_probabilities)
    dirty_idx = (res.T[0] <= 24000)
    clean_idx = (res.T[0] > 24000)
    new_weights = torch.tensor([0 if i < 0.9 else i for i in clean_probabilities])
    print((new_weights[dirty_idx] > 0).sum())
    print((new_weights[clean_idx] > 0).sum() / len(new_weights))
    selected_by_threshold = index[:int(cos_sim_threshold * len(index))]
    print((selected_by_threshold < 24000).sum())
    # train_index = new_weights.nonzero().T
    # train.append(index[train_index])
    # clean_proportion = ((new_weights[clean_idx] > 0).sum() / len(new_weights))
    # poisoned_proportion = ((new_weights[dirty_idx] > 0).sum() / len(new_weights[dirty_idx]))
    # clean.append(clean_proportion)
    # poison.append(poisoned_proportion)
    # meang.append(clean_mean)

    # new_weights_stronger = torch.tensor([0 if i < 0.8 else i for i in clean_probabilities])

    # poisoner.append((new_weights_stronger[dirty_idx] > 0).sum() / len(new_weights_stronger[dirty_idx]))
    # cleaner.append((new_weights_stronger[clean_idx] > 0).sum() / len(new_weights_stronger))

    
# new_weights_stronger = torch.tensor([0 if i < 0.8 else i for i in clean_probabilities])
# print((new_weights_stronger[dirty_idx] > 0).sum() / len(new_weights_stronger[dirty_idx]))
# print((new_weights_stronger[clean_idx] > 0).sum() / len(new_weights_stronger))

# print((torch.tensor(prob) * dirty_idx.detach().cpu()).sum()/8500)
# print((torch.tensor(prob) * clean_idx.detach().cpu()).sum()/len(prob))



tensor(55)
tensor(0.1778)
tensor(33)
